In [26]:
import gymnasium as gym
env = gym.make("FrozenLake-v1")  # 创建环境
_ = env.reset(seed=0)
env = env.unwrapped  # 解封装才能访问状态转移矩阵P
env.render()  # 环境渲染,通常是弹窗显示或打印出可视化的环境

holes = set()
ends = set()
for s in env.P:  ##  遍历环境的所有状态
    for a in env.P[s]:  ##  遍历状态下所有的动作
        for s_ in env.P[s][a]:  ##  遍历每个动作对应的转移情况
            if s_[2] == 1.0:  # 获得奖励为1,代表是目标
                ends.add(s_[1]) ##  终止标号的呢
            if s_[3] == True:   ##  是否洞
                holes.add(s_[1])  ## 标号
holes = holes - ends
print("冰洞的索引:", holes)
print("目标的索引:", ends)

## 共有三个动作的，每个动作有三种转移可能
## 每个动作都有三种状态转移的可能，和上面的悬崖有些区别的，上面的悬崖只有一种可能
## 导致这种情况的reason，就是冰面太滑了，可能滑到其他地方去了
for a in env.P[14]:  # 查看目标左边一格的状态转移信息
    print(env.P[14][a])

# SFFF
# FHFH
# FFFH
# HFFG
# 冰洞的索引: {11, 12, 5, 7}
# 目标的索引: {15}
# [(0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False),
#  (0.3333333333333333, 14, 0.0, False)]
# [(0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False),
#  (0.3333333333333333, 15, 1.0, True)]
# [(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True),
#  (0.3333333333333333, 10, 0.0, False)]
# [(0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False),
#  (0.3333333333333333, 13, 0.0, False)]

冰洞的索引: {11, 12, 5, 7}
目标的索引: {15}
[(0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False)]
[(0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True)]
[(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False)]
[(0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False)]


In [29]:
# 这个动作意义是Gym库针对冰湖环境事先规定好的
action_meaning = ['<', 'v', '>', '^']   ## 上下左右的标识符
theta = 1e-5   ## 当绝对值差值小于阀值时，停止迭代的呢
gamma = 0.9    ## γ值
agent = PolicyIteration(env, theta, gamma)  ## 实例化策略迭代
agent.policy_iteration()      ## 迭代
print_agent(agent, action_meaning, [5, 7, 11, 12], [15]) ## 输出

# 策略评估进行25轮后完成
# 策略提升完成
# 策略评估进行58轮后完成
# 策略提升完成
# 状态价值：
#  0.069  0.061  0.074  0.056
#  0.092  0.000  0.112  0.000
#  0.145  0.247  0.300  0.000
#  0.000  0.380  0.639  0.000
# 策略：
# <ooo ooo^ <ooo ooo^
# <ooo **** <o>o ****
# ooo^ ovoo <ooo ****
# **** oo>o ovoo EEEE

策略评估进行25轮后完成
策略提升完成
策略评估进行58轮后完成
策略提升完成
状态价值：
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 


In [8]:
action_meaning = ['<', 'v', '>', '^']   ## 上下左右的标识符
theta = 1e-5   ## 当绝对值差值小于阀值时，停止迭代的呢
gamma = 0.9    ## γ值
agent = ValueIteration(env, theta, gamma)  ## 实例化价值迭代
agent.value_iteration()      ## 迭代
print_agent(agent, action_meaning, [5, 7, 11, 12], [15]) ## 输出

# 价值迭代一共进行60轮
# 状态价值：
#  0.069  0.061  0.074  0.056
#  0.092  0.000  0.112  0.000
#  0.145  0.247  0.300  0.000
#  0.000  0.380  0.639  0.000
# 策略：
# <ooo ooo^ <ooo ooo^
# <ooo **** <o>o ****
# ooo^ ovoo <ooo ****
# **** oo>o ovoo EEEE

价值迭代一共进行60轮
状态价值：
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 
